# Adapted from tutorial nemo 3

In [ ]:
import pathlib

from parcels import FieldSet, Field, ParticleSet, JITParticle, AdvectionRK4_3D

from parcels import logger, XarrayDecodedFilter
logger.addFilter(XarrayDecodedFilter())  # Add a filter for the xarray decoding warning

# Subset, lower left corner of my grid
enatl_data_path = pathlib.Path('day_1_5')

u_files = sorted(enatl_data_path.glob('*1d_vozocrtx*'))
v_files = sorted(enatl_data_path.glob('*1d_vomecrty*'))
w_files = sorted(enatl_data_path.glob('*1d_vovecrtz*'))

assert len(u_files) == len(w_files)
assert len(u_files) == len(v_files)

enatl_mesh_mask_file = enatl_data_path / 'mesh_mask_eNATL60FARSHE_3.6.nc'

The given data files are on u-, v, and, w-points respectively. The data is output from a nemo 3.6 model run, [[https://github.com/ocean-next/eNATL60]]

According to the [[NEMO 3D tutorial|https://nbviewer.jupyter.org/github/OceanParcels/parcels/blob/master/parcels/examples/tutorial_nemo_3D.ipynb]], the f-points should be given.

In [ ]:
filenames_as_in_tutorial = {
    'U': {'lon': enatl_mesh_mask_file, 'lat': enatl_mesh_mask_file, 'depth': w_files[0], 'data': u_files},
    'V': {'lon': enatl_mesh_mask_file, 'lat': enatl_mesh_mask_file, 'depth': w_files[0], 'data': v_files},
    'W': {'lon': enatl_mesh_mask_file, 'lat': enatl_mesh_mask_file, 'depth': w_files[0], 'data': w_files},
}

variables = {
    'U': 'vozocrtx',
    'V': 'vomecrty',
    'W': 'vovecrtz',
}

dimensions_as_in_tutorial = {
    'U': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'},
    'V': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'},
    'W': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'},
}


In [ ]:
fieldset_as_in_tutorial = FieldSet.from_nemo(filenames_as_in_tutorial, variables, dimensions_as_in_tutorial)

In [ ]:
fieldset_as_in_tutorial.U.show()

But also, from the documentation string from ``parcels.FieldSet.from_c_grid_dataset

       To interpolate U, V velocities on the C-grid, Parcels needs to read the f-nodes,
       which are located on the corners of the cells.
       (for indexing details: https://www.nemo-ocean.eu/doc/img360.png )
       In 3D, the depth is the one corresponding to W nodes.

# Question
So, if I already have my variables o the fieldset, why don't I do the following?

In [ ]:
filenames_assuming_c_grid = {
    'U': {'lon': enatl_mesh_mask_file, 'lat': enatl_mesh_mask_file, 'depth': u_files[0], 'data': u_files},
    'V': {'lon': enatl_mesh_mask_file, 'lat': enatl_mesh_mask_file, 'depth': v_files[0], 'data': v_files},
    'W': {'lon': enatl_mesh_mask_file, 'lat': enatl_mesh_mask_file, 'depth': w_files[0], 'data': w_files},
}

variables = {
    'U': 'vozocrtx',
    'V': 'vomecrty',
    'W': 'vovecrtz',
}

dimensions_assuming_c_grid = {
    'U': {'lon': 'glamu', 'lat': 'gphiu', 'depth': 'depthu', 'time': 'time_counter'},
    'V': {'lon': 'glamv', 'lat': 'gphiv', 'depth': 'depthv', 'time': 'time_counter'},
    'W': {'lon': 'glamw', 'lat': 'gphiw', 'depth': 'depthw', 'time': 'time_counter'},
}


In [ ]:
fieldset_c_grid_points = FieldSet.from_nemo(filenames_assuming_c_grid, variables, dimensions_assuming_c_grid)

Now I want to add temperature and salinity. In the [[NEMO 3D tutorial|https://nbviewer.jupyter.org/github/OceanParcels/parcels/blob/master/parcels/examples/tutorial_nemo_3D.ipynb]], they exemplify this by adding scale factor. They are given on the t-points. Why should these variables be given on their t-points and not on the f-points?

In [ ]:
vosalinity_files = sorted(enatl_data_path.glob('*1d_vosaline*'))
votemper_files =  sorted(enatl_data_path.glob('*1d_votemper*'))

# regardless of naming,
dimensions_t_points = dict(lon='nav_lon', lat='nav_lat', depth='deptht', time='time_counter')

vosalinity = Field.from_netcdf(vosalinity_files, ('S', 'vosaline'), dimensions_t_points)
votemper = Field.from_netcdf(votemper_files, ('T', 'votemper'), dimensions_t_points)

In [ ]:
# In case I happened to run a parcel execute I reinitialise fildset_as_in_tutorial
fieldset_as_in_tutorial = FieldSet.from_nemo(filenames_as_in_tutorial, variables, dimensions_as_in_tutorial)

In [ ]:
fieldset_as_in_tutorial.add_field(vosalinity)
fieldset_as_in_tutorial.add_field(votemper)

In [ ]:
fieldset_as_in_tutorial.T.show()

In [ ]:
fieldset_as_in_tutorial.S.show()